In [ ]:
import os
import numpy as np
import music21
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.utils import to_categorical

# Set random seed for reproducibility
np.random.seed(42)

# Load MIDI files from a directory
def load_midi_files(directory):
    midi_files = []
    for file in os.listdir(directory):
        if file.endswith(".midi"):
            midi_files.append(os.path.join(directory, file))
    return midi_files

# Parse MIDI files and extract musical sequences
def parse_midi_files(midi_files):
    notes = []
    for file in midi_files:
        midi = music21.converter.parse(file)
        notes_to_parse = None
        parts = music21.instrument.partitionByInstrument(midi)
        if parts:  # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, music21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, music21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

# Prepare input and output sequences for the model
def prepare_sequences(notes, sequence_length=100):
    note_to_int = {note: number for number, note in enumerate(sorted(set(notes)))}
    int_to_note = {number: note for note, number in note_to_int.items()}
    
    sequences = []
    next_notes = []
    for i in range(len(notes) - sequence_length):
        sequences.append([note_to_int[n] for n in notes[i:i + sequence_length]])
        next_notes.append(note_to_int[notes[i + sequence_length]])
    
    if len(sequences) == 0 or len(next_notes) == 0:
        return None, None, note_to_int, int_to_note
    
    X = np.reshape(sequences, (len(sequences), sequence_length, 1)) / float(len(note_to_int))
    y = to_categorical(next_notes, num_classes=len(note_to_int))
    
    return X, y, note_to_int, int_to_note

# Create the RNN model
def create_model(input_shape, num_classes):
    model = Sequential([
        LSTM(256, input_shape=input_shape, return_sequences=True),
        Dropout(0.3),
        LSTM(512, return_sequences=True),
        Dropout(0.3),
        LSTM(256),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Generate music sequences using the trained model
def generate_music(model, note_to_int, int_to_note, sequence_length=100, num_notes=500):
    start_index = np.random.randint(0, len(notes) - sequence_length - 1)
    pattern = notes[start_index:start_index + sequence_length]
    generated_notes = []

    for _ in range(num_notes):
        input_sequence = np.array([note_to_int[note] for note in pattern]).reshape(1, sequence_length, 1) / float(len(note_to_int))
        prediction = model.predict(input_sequence, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        generated_notes.append(result)

        pattern.append(result)
        pattern = pattern[1:]

    return generated_notes

# Directory containing MIDI files
midi_directory = "midi_files"

# Load and parse MIDI files
midi_files = load_midi_files(midi_directory)
if not midi_files:
    raise ValueError("No MIDI files found in the specified directory.")

notes = parse_midi_files(midi_files)
if not notes:
    raise ValueError("No notes parsed from the MIDI files.")

# Prepare input and output sequences
sequence_length = 100
X, y, note_to_int, int_to_note = prepare_sequences(notes, sequence_length)
if X is None or y is None:
    raise ValueError("No sequences generated from the notes. Check the sequence length and input data.")

# Create and train the model
input_shape = (X.shape[1], X.shape[2])
num_classes = y.shape[1]
model = create_model(input_shape, num_classes)
model.fit(X, y, epochs=50, batch_size=64)

# Generate music sequence
generated_notes = generate_music(model, note_to_int, int_to_note)

# Create a stream object and add notes
output_stream = music21.stream.Stream()
for note in generated_notes:
    if '.' in note:
        chord_notes = note.split('.')
        chord = music21.chord.Chord(chord_notes)
        output_stream.append(chord)
    else:
        output_stream.append(music21.note.Note(note))

# Write the stream to a MIDI file
output_stream.write('midi', fp='generated_music.mid')
